In [1]:
import pandas as pd
%matplotlib inline
#GRO import
GRO = pd.read_excel('https://recovery.smithschool.ox.ac.uk/wp-content/uploads/2021/05/20210524-Global-Recovery-Observatory-publicv2.xlsx', sheet_name="COVID-19 Measures",
                  na_values=[".."], usecols={'Country', 'Total Value, USD (billions)', 'Clean archetype?', 'Recovery?'})
index = GRO[GRO['Recovery?'] != 1].index
rec = GRO.drop(index, inplace=False)
rec = rec.groupby(["Country"], as_index=False).sum()
rec = rec.drop(columns=['Recovery?', 'Clean archetype?'])

index2 = GRO[GRO['Clean archetype?'] != 1].index
greenrec = GRO.drop(index2, inplace=False)
index = greenrec[greenrec['Recovery?'] != 1].index
greenrec.drop(index, inplace=True)
greenrec = greenrec.groupby(["Country"], as_index=False).sum()
greenrec = greenrec.drop(columns=['Recovery?', 'Clean archetype?'])

In [2]:
merge = greenrec.merge(rec, how='left', left_on='Country', right_on='Country')
merge['Percent'] = (merge['Total Value, USD (billions)_x'] / merge['Total Value, USD (billions)_y']) * 100
merge.rename(columns = {"Total Value, USD (billions)_x" : "Green recovery spending", "Total Value, USD (billions)_y" : "Recovery spending"}, inplace=True)
merge = merge.round({"Percent" : 2})
merge = merge[['Country', 'Green recovery spending', 'Percent']]
merge

,Country,Green recovery spending,Percent
0,Argentina,0.00,0.00
1,Australia,2.30,1.76
2,Austria,0.38,23.75
3,Bangladesh,0.55,87.30
4,Brazil,0.28,51.85
5,Burkina Faso,0.00,NaN
6,Canada,21.67,55.99
7,Chile,0.45,14.38
8,China,51.88,12.14
9,Colombia,0.15,46.88


In [ ]:
#renewable energy consumption import
df = pd.read_excel(r'C:\Users\ebloom\Desktop\IRENA.xlsx',
                  na_values=["0", ".."])
df = df[df['Year'] == 2018]
df = df[df['Technology/source'] == 'GrandTotal']
df = df[df['Category'] == 'Final Consumption-Total']
df = df.round(2)
df.loc[df['Country']=='United States of America']
df.at[11368, 'Country'] = 'United States'
GROmerge = merge.merge(df, how='left', left_on='Country', right_on='Country')
GROmerge = GROmerge[['Country', 'Percent', 'Green recovery spending', 'TJ']]
GROmerge.rename(columns = {"TJ" : "Renewable energy consumption (TJ)"}, inplace=True)
GROmerge

In [ ]:
import pycountry
#deforestation import
df = pd.read_csv(r'C:\Users\ebloom\Desktop\Global_Annual_Treecover_Loss.csv', usecols={"iso", "umd_tree_cover_loss__year", "umd_tree_cover_loss__ha"},
                  na_values=["0", ".."])
df.rename(columns = {"umd_tree_cover_loss__year" : "Year", "umd_tree_cover_loss__ha" : "Tree Cover Loss (ha)"}, inplace=True)
df = df.round({'Tree Cover Loss (ha)' : 2})
df = pd.pivot_table(df, values='Tree Cover Loss (ha)', index=['iso'], columns=['Year'])
df.rename(columns={2018 : '2018', 2019 : '2019'},inplace=True)
df = df[['2018', '2019']]

countries = {}
for country in pycountry.countries:
    countries[country.alpha_3] = country.name
##print(countries)
df['Country'] = [countries.get(code) for code in df.index]
df.at['VNM', 'Country'] = "Vietnam"
df.at['KOR', 'Country'] = "South Korea"
df

In [ ]:
GROmerge = GROmerge.merge(df, how='left', left_on='Country', right_on='Country')
GROmerge.rename(columns = {"2019" : "Annual tree cover loss (ha)"}, inplace=True)
GROmerge = GROmerge[['Country', 'Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)']]
#GROmerge = GROmerge.drop(columns={'2018'})
GROmerge

In [ ]:
#natural resource rents import

nr = pd.read_excel(r'C:\Users\ebloom\Desktop\Data_Extract_From_Adjusted_Net_Savings.xlsx',
                   usecols=["Country Name", "Country Code", "2017 [YR2017]", "2018 [YR2018]", "2019 [YR2019]"],
                  na_values=["0", ".."])
nr.rename(columns = {"2017 [YR2017]" : "2017", "2018 [YR2018]" : "2018", "2019 [YR2019]" : "2019"}, inplace=True)
nr = nr.round({'2017' : 2, '2018' : 2, '2019' : 2})
nr = nr.drop(nr.index[264:269])
nr = nr.drop(nr.index[0:47])
nr.at[152, 'Country Name'] = "South Korea"
nr

In [ ]:
GROmerge = GROmerge.merge(nr, how='left', left_on='Country', right_on='Country Name')
GROmerge = GROmerge[['Country', 'Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)', '2019', 'Country Code']]
GROmerge.rename(columns = {"2019" : "Natural resource rents as % GDP"}, inplace=True)
GROmerge

In [ ]:
#fossil fuel subs import
df = pd.read_excel(r'C:\Users\ebloom\Desktop\FossilFuelSubsidiesTracker.xlsx', sheet_name='fulldata',
                  na_values=["0.00", ".."])
df = df[df['Year'] == 2019]
df = df.groupby(['Country']).sum()
df = df.drop(columns={'Year'})
df['% share of GDP'] = df['% share of GDP'] * 100
df = df.round(2)
df = df.reset_index()
df

In [ ]:
df[df['Country'] == 'Korea']
df.at[42, 'Country'] = "South Korea"
df[df['Country'] == 'Viet Nam']
df.at[80, 'Country'] = "Vietnam"
df[df['Country'] == "People's Republic of China"]
df.at[57, 'Country'] = "China"

In [ ]:
GROmerge = GROmerge.merge(df, how='left', left_on='Country', right_on='Country')
GROmerge.rename(columns = {"% share of GDP" : "Fossil fuel subsidies as % GDP", "USD nominal per Capita": "Fossil fuel subsidies USD per Capita"}, inplace=True)
GROmerge

In [ ]:
#CO2 data import

df = pd.read_excel('https://iea.blob.core.windows.net/assets/6296011e-4fed-40c3-95a0-1367a4c4484a/CO2Highlights2020.xls',
                   'CO2-POP', header=21, usecols=[0, 47, 48, 49], na_values='..')
df.columns = df.columns.astype(str)
df = df.round({'2017' : 2, '2018' : 2, '2019' : 2})
df = df[df['Region/Country/Economy'].str.contains('OECD')==False]
df = df.drop(df.index[169:172])
df = df.sort_values('Region/Country/Economy', ascending=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
df['Code'] = [countries.get(country) for country in df['Region/Country/Economy']]
df

In [ ]:
df.at[18, 'Code'] = 'BOL'
df.at[29, 'Code'] = 'TWN'
df.at[37, 'Code'] = 'CZE'
df.at[40, 'Code'] = 'COD'
df.at[76, 'Code'] = 'IRN'
df.at[84, 'Code'] = 'KOR'
df.at[119, 'Code'] = 'CHN'
df.at[125, 'Code'] = 'MDA'
df.at[133, 'Code'] = 'SVK'
df.at[143, 'Code'] = 'SYR'
df.at[154, 'Code'] = 'TZA'
df.at[158, 'Code'] = 'VEN'

In [ ]:
df = df.dropna(subset=['Code'])
df = df.rename(columns={'Region/Country/Economy' : 'Country'})
df.at[84, 'Country'] = 'South Korea'
df.at[119, 'Country'] = 'China'
df.at[159, 'Country'] = 'Vietnam'
df

In [ ]:
GROmerge = GROmerge.merge(df, how='left', left_on='Country', right_on='Country')
GROmerge.rename(columns = {"2018" : "CO2 emissions tons per capita"}, inplace=True)
GROmerge

In [ ]:
GROmerge = GROmerge[['Country', 'Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)',
                   'Natural resource rents as % GDP', 'Fossil fuel subsidies as % GDP', 'Fossil fuel subsidies USD per Capita', 'CO2 emissions tons per capita',
                   'Country Code']]
GROmerge

In [ ]:
# air pollution import

df = pd.read_excel(r'C:\Users\ebloom\Desktop\AAP.xlsx', sheet_name='latest availble PM25 (measured)', header=2,
                  na_values=["0", ".."])
df = df[df['Database version (year)'] == 2018]
df = df.drop(columns={'Region', 'Annual mean, ug/m3', 'note on converted PM10', 'Temporal coverage', 'status'})
df.loc[(df.iso3=='KOR', 'Country')] = 'South Korea'
df.loc[(df.iso3=='USA', 'Country')] = 'United States'
df.loc[(df.iso3=='VNM'), 'Country'] = 'Vietnam'
df = df[['iso3', 'Country', 'City/Town', 'Annual mean, ug/m3.1']]
df

In [ ]:
citypop = pd.read_csv(r'C:\Users\ebloom\Desktop\Citypop.csv',
                  na_values=["0", ".."])
citypop = citypop[['Name', 'Country', 'Population']]
citypop = citypop.sort_values(by='Country', ascending=True)
citypop.reset_index(inplace=True, drop=True)
ind = citypop.groupby(['Country'])['Population'].idxmax()
biggest = citypop.loc[ind]
biggest
#find most populous cities for each country in dataset

In [ ]:
#what needs imputing, what should be there but difference in nomenclature
df[df['Country'] == 'Australia']
#Arg not in pollution data
#biggest city Melbourne only 2016 data - needs to be imputed
df[df['Country'] == 'Austria']
df.at[72, 'City/Town'] = 'Vienna'
df[df['Country'] == 'Colombia']
#biggest city Bogota not in data at all - needs to be imputed
df[df['Country'] == 'Dominican Republic']
#DR not in pollution data
#Iraq and Honduras not in at all
df[df['Country'] == 'Ireland']
df.at[1376, 'City/Town'] = 'Dublin'
df[df['City/Town'] == 'Roma']
df.at[1626, 'City/Town'] = 'Rome'
#Kenya and MUS not in data at all
#biggest city in Pakistan not in 2018 data, Karachi only 2016 needs to be imputed
biggest[biggest['Country'] == 'Pakistan']
df[df['City/Town'] == 'Warszawa']
df.at[1907, 'City/Town'] = 'Warsaw'
df[df['City/Town'] == 'New York']
df.at[2393, 'City/Town'] = 'New York City'
#no data for Jamaica at all
df[df['Country'] == 'Jamaica']

In [ ]:
pop = biggest.merge(df, how='left', left_on='Name', right_on='City/Town')
pop = pop.dropna(subset={'Annual mean, ug/m3.1'})
pop
#pollution in the world's biggest cities

In [ ]:
import numpy as np
#Havana, Dublin, New York City
#Kingston, Jamaica should not match Kingston Canada, should be NA
GROmerge = GROmerge.merge(pop, how='left', left_on='Country', right_on='Country_x')
GROmerge = GROmerge.drop([5, 10, 13, 14, 44, 47])
GROmerge.reset_index(drop=True, inplace=True)
GROmerge = GROmerge[['Country', 'Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)',
                   'Natural resource rents as % GDP', 'Fossil fuel subsidies as % GDP', 'Fossil fuel subsidies USD per Capita', 'CO2 emissions tons per capita',
                   'Annual mean, ug/m3.1', 'Country Code']]
GROmerge.at[21, 'Annual mean, ug/m3.1'] = np.nan
GROmerge.rename(columns = {"Annual mean, ug/m3.1" : "Air pollution (annual mean ug/m3 <PM2.5)"}, inplace=True)
GROmerge
#here is where I could potentially make them into rates

In [ ]:
#Original row is Argentina, go through all other rows, start with Australia
#Subtract Argentina’s TJ from Australia’s
#Take absolute value
#With list of differences after gone through all columns, take average across them (careful of unit) – becomes Australia’s score
#include green recovery spending cols????
#isolate only numeric columns
diffset = GROmerge[['Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)',
                   'Natural resource rents as % GDP', 'Fossil fuel subsidies as % GDP', 'Fossil fuel subsidies USD per Capita', 'CO2 emissions tons per capita',
                   'Air pollution (annual mean ug/m3 <PM2.5)']]
#set up
difference = diffset
difference = difference.drop(range(1,42))
for name, value in difference.iteritems():
    difference.at[0, name] = np.nan

#diffresult = diffset.diff(periods=1)
#difference = difference.append(diffresult[diffresult.index == 1], ignore_index=True)
n=1

for index, row in diffset.iterrows():
    diffresult = diffset.diff(periods=n)
    difference = difference.append(diffresult[diffresult.index == n], ignore_index=True)
    n = n+1
    
difference = np.absolute(difference)
#air poll for Arg Nan - does this mean the unknown columns can not factor into NN? all become NA
#normalise data with maximum absolute scaling https://www.geeksforgeeks.org/data-normalization-with-pandas/
difference_norm = difference
for column in difference_norm.columns:
    difference_norm[column] = difference_norm[column]  / difference_norm[column].abs().max()
difference_norm

In [ ]:
difference_norm['Score'] = difference_norm.mean(axis=1)
difference_norm
##difference_norm['Score'].min()
##difference_norm[difference_norm['Score'] == 0.04129648718215812]
##row = GROmerge[GROmerge.index == 30]
##airpoll = row.iloc[0]['Air pollution (annual mean ug/m3 <PM2.5)']
##GROmerge['Air pollution_imputed'] = np.nan
##GROmerge.at[0, 'Air pollution_imputed'] = airpoll
##GROmerge
#Peru closest to Arg
#closest to 0 is nearest neighbour

In [ ]:
#again for Australia
diffset = GROmerge[['Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)',
                   'Natural resource rents as % GDP', 'Fossil fuel subsidies as % GDP', 'Fossil fuel subsidies USD per Capita', 'CO2 emissions tons per capita',
                   'Air pollution (annual mean ug/m3 <PM2.5)']]
#set up
difference = diffset
difference = difference.drop(range(1,42))
for name, value in difference.iteritems():
    difference.at[0, name] = np.nan


In [ ]:
n=1
m=2

for index, row in diffset.iterrows():
    diffresult = diffset.diff(periods=n)
    difference = difference.append(diffresult[diffresult.index == m], ignore_index=True)
    n = n+1
    m = m+1
difference

In [ ]:
difference.index = range(1,42)
diffresult = diffset.diff(periods=-1)
difference = difference.append(diffresult[diffresult.index == 0])

In [ ]:
difference = np.absolute(difference)
#normalise data with maximum absolute scaling https://www.geeksforgeeks.org/data-normalization-with-pandas/
difference_norm = difference
for column in difference_norm.columns:
    difference_norm[column] = difference_norm[column]  / difference_norm[column].abs().max()
difference_norm

In [ ]:
difference_norm['Score'] = difference_norm.mean(axis=1)
difference_norm['Score'].min()


In [ ]:
difference_norm[difference_norm['Score'] == 0.24595686197296526]
#note: renewable energy cons not even taken into account here

In [ ]:
row = GROmerge[GROmerge.index == 41]
airpoll = row.iloc[0]['Air pollution (annual mean ug/m3 <PM2.5)']
renewcons = row.iloc[0]['Renewable energy consumption (TJ)']
GROmerge.at[1, 'Air pollution_imputed'] = airpoll
GROmerge['Renewable energy_imputed'] = np.nan
GROmerge.at[1, 'Renewable energy_imputed'] = renewcons
GROmerge
#USA closest to Aus
#closest to 0 is nearest neighbour

In [ ]:
GROmerge[GROmerge.index == 41]
GROmerge = GROmerge.drop(columns= ['Air pollution_imputed', 'Renewable energy_imputed'])
#what to do when nearest neighbour also has a null in the column you are looking to impute? copy from second closest neighbour?

In [ ]:
#normalise columns so that the columns are on same scale
#here, make sure all columns have a mean of 0 and std deviation of 1
#set mean to 0 by finding the mean of a column, and then subtract the column's mean from every value in the column
#set std deviation to 1 by dividing every value in the col by the std deviation
#i think ive standardise but should have normalised here

#find numeric cols
diffset = GROmerge[['Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)',
                   'Natural resource rents as % GDP', 'Fossil fuel subsidies as % GDP', 'Fossil fuel subsidies USD per Capita', 'CO2 emissions tons per capita',
                   'Air pollution (annual mean ug/m3 <PM2.5)']]

normal = (diffset - diffset.mean()) / diffset.std()
normal

In [ ]:
import math
#that is manual example - try k nearest neighbours
#https://www.dataquest.io/blog/k-nearest-neighbors-in-python/

#first, take euclidian distance using function from dataquest based on ED formula
diffset = GROmerge[['Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)',
                   'Natural resource rents as % GDP', 'Fossil fuel subsidies as % GDP', 'Fossil fuel subsidies USD per Capita', 'CO2 emissions tons per capita',
                   'Air pollution (annual mean ug/m3 <PM2.5)']]

selected_country = GROmerge[GROmerge['Country'] == 'Argentina'].iloc[0]
#for each column, take diff of values in two rows, square them, and then add difference to total (sum squared differences), then square root the sum

def euclidean_distance(x):
    inner_val = 0
    for k in diffset:
        if np.isnan(x[k]):
            inner_val += 0
            #print("x:", x[k])
            #print("selected:", selected_country[k])
            #print("in:", inner_val)
        elif np.isnan(selected_country[k]):
            inner_val += 0
            #print("x:", x[k])
            #print("selected:", selected_country[k])
            #print("in:", inner_val)
        else:
            result = (x[k] - selected_country[k]) ** 2
            inner_val += result
            #print("x:", x[k])
            #print("selected:", selected_country[k])
            #print("res:", result)
            #print("in:", inner_val)
    #print("new row")
    return math.sqrt(inner_val)
#make sure to ignore nan values or the inner val will become nan and reset - both for selected country row and other rows?
#skip cols with nan
arg_distance = GROmerge.apply(euclidean_distance, axis=1)
arg_distance
#Argentinian euclidean distance from all other rows

In [ ]:
from scipy.spatial import distance
##using scipy library now rather than simple function above
#We can use the distance.euclidean function from scipy.spatial, a much faster way to calculate euclidean distance.
normal.fillna(0, inplace=True)
arg_normal = normal[GROmerge['Country'] == 'Argentina']
euc_dist = normal.apply(lambda row: distance.euclidean(row, arg_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
#distances
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[int(second_nearest)]['Country']
winner
#algorithm correct? find nearest neighbour, take the values from that row for those missing in original row
##this way, DR second too, Honduras, Philippines (lots of Nans), all around 0.05 other way too, Italy kind of random, then Peru
##closest for Arg is Mexico in this case, Mexico row 25 was third other way (manual) BUT only one with no Nans, the others had quite a few (DR 10, Peru 30)

In [ ]:
aus_normal = normal[GROmerge['Country'] == 'Australia']
euc_dist = normal.apply(lambda row: distance.euclidean(row, aus_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner
#also US like other way - not anymore now that we have renewable energy cons added for US

In [ ]:
bng_normal = normal[GROmerge['Country'] == 'Bangladesh']
euc_dist = normal.apply(lambda row: distance.euclidean(row, bng_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
can_normal = normal[GROmerge['Country'] == 'Canada']
euc_dist = normal.apply(lambda row: distance.euclidean(row, can_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
col_normal = normal[GROmerge['Country'] == 'Colombia']
euc_dist = normal.apply(lambda row: distance.euclidean(row, col_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
dr_normal = normal[GROmerge['Country'] == 'Dominican Republic']
euc_dist = normal.apply(lambda row: distance.euclidean(row, dr_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
ghana_normal = normal[GROmerge['Country'] == 'Ghana']
euc_dist = normal.apply(lambda row: distance.euclidean(row, ghana_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
hon_normal = normal[GROmerge['Country'] == 'Honduras']
euc_dist = normal.apply(lambda row: distance.euclidean(row, hon_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
india_normal = normal[GROmerge['Country'] == 'India']
euc_dist = normal.apply(lambda row: distance.euclidean(row, india_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
irq_normal = normal[GROmerge['Country'] == 'Iraq']
euc_dist = normal.apply(lambda row: distance.euclidean(row, irq_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
jamaica_normal = normal[GROmerge['Country'] == 'Jamaica']
euc_dist = normal.apply(lambda row: distance.euclidean(row, jamaica_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
jpn_normal = normal[GROmerge['Country'] == 'Japan']
euc_dist = normal.apply(lambda row: distance.euclidean(row, jpn_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
ken_normal = normal[GROmerge['Country'] == 'Kenya']
euc_dist = normal.apply(lambda row: distance.euclidean(row, ken_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
mus_normal = normal[GROmerge['Country'] == 'Mauritius']
euc_dist = normal.apply(lambda row: distance.euclidean(row, mus_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
pakistan_normal = normal[GROmerge['Country'] == 'Pakistan']
euc_dist = normal.apply(lambda row: distance.euclidean(row, pakistan_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
panama_normal = normal[GROmerge['Country'] == 'Panama']
euc_dist = normal.apply(lambda row: distance.euclidean(row, panama_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
peru_normal = normal[GROmerge['Country'] == 'Peru']
euc_dist = normal.apply(lambda row: distance.euclidean(row, peru_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
phil_normal = normal[GROmerge['Country'] == 'Philippines']
euc_dist = normal.apply(lambda row: distance.euclidean(row, phil_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
sen_normal = normal[GROmerge['Country'] == 'Senegal']
euc_dist = normal.apply(lambda row: distance.euclidean(row, sen_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
kor_normal = normal[GROmerge['Country'] == 'South Korea']
euc_dist = normal.apply(lambda row: distance.euclidean(row, kor_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
switz_normal = normal[GROmerge['Country'] == 'Switzerland']
euc_dist = normal.apply(lambda row: distance.euclidean(row, switz_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
thai_normal = normal[GROmerge['Country'] == 'Thailand']
euc_dist = normal.apply(lambda row: distance.euclidean(row, thai_normal), axis=1)
distances = pd.DataFrame(data={"Distance": euc_dist, "Index": euc_dist.index})
distances.sort_values("Distance", inplace=True)
second_nearest = distances.iloc[1]["Index"]
winner = GROmerge.loc[second_nearest]['Country']
winner

In [ ]:
#generate score
#set up test and train sets by using random samplings - shuffle the index and pick rows, otherwise predicting and training on the same data - overfit
import random
from numpy.random import permutation

GROmergez = GROmerge.fillna(0, inplace=False)
shuffled = permutation(GROmergez.index)
#no more than 1/3 of countries in a test set, here testfloor is 14
testfloor = math.floor(len(GROmergez)/3)
#take first third of randomly shuffled indices
testset = GROmergez.loc[shuffled[1:testfloor]]
#train set becomes rest of data
trainset = GROmergez.loc[shuffled[testfloor:]]
trainset

In [ ]:
#using scikitlearn for k nearest neighbours - does normalisation and finding distance score automatically
#using regressor (continuous data)
#select columns to predict with
xcols = ['Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)',
                   'Natural resource rents as % GDP', 'Fossil fuel subsidies as % GDP', 'Fossil fuel subsidies USD per Capita', 'CO2 emissions tons per capita']
#select column we want to predict
ycol = ['Air pollution (annual mean ug/m3 <PM2.5)']

from sklearn.neighbors import KNeighborsRegressor
#create k nearest neighbours model, here with 5 closest neighbours (k=5), user defined constant
nearest = KNeighborsRegressor(n_neighbors=5)
#fit the model on the training data
nearest.fit(trainset[xcols], trainset[ycol])
#make point predictions on the test set using the fit model
pp = nearest.predict(testset[xcols])

In [ ]:
pp
##we think for the test set, these are the air pollution vals?

In [ ]:
#https://scikit-learn.org/stable/modules/neighbors.html
actual = testset[ycol]
#compute mean squared error using this formula
#for each row in test set, subtract actual values from predicted values, square the result and then add them together, divide by number of rows in predicted set
#basically finding average squared difference yeah
pp - actual
#error = (((pp - actual) ** 2).sum()) / len(pp)
#error

In [ ]:
from sklearn.impute import KNNImputer
#missing values replaced by mean value of 5 nearest neighbours measured by Euc dist as distance metric

#scale values to have values between 0 and 1 using Scikit learn's MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

#several key principles there already above
#find numeric cols
diffset = GROmerge[['Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)',
                   'Natural resource rents as % GDP', 'Fossil fuel subsidies as % GDP', 'Fossil fuel subsidies USD per Capita', 'CO2 emissions tons per capita',
                   'Air pollution (annual mean ug/m3 <PM2.5)']]
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(diffset), columns = diffset.columns)
df
#multivariate approach to imputation now, not just taking average for that column (univariate)

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html
#looks for missing values in all columns, uses others to fill them
imputer = KNNImputer(n_neighbors=5)
#fit to data, then transform it
df = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)

In [ ]:
df

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler.inverse_transform
df = pd.DataFrame(scaler.inverse_transform(df), columns = df.columns)
df

In [ ]:
#put these back onto the GROmerge in new cols under per list

In [ ]:
per_list = []

for name, values in GROmerge.iteritems():
    narows = GROmerge[GROmerge[name].isna()]
    ## number of NA rows divided by total no of rows
    narows['Country'].count()
    per_imputed = narows['Country'].count() / GROmerge['Country'].count()
    per_list.append(per_imputed)

per_imp = pd.Series(per_list, index=['Country', 'Green recovery spending', 'Percent', 'Renewable energy consumption (TJ)', 'Annual tree cover loss (ha)',
                   'Natural resource rents as % GDP', 'Fossil fuel subsidies as % GDP', 'Fossil fuel subsidies USD per Capita', 'CO2 emissions tons per capita',
                   'Air pollution (annual mean ug/m3 <PM2.5)', 'Country Code'])
GROmerge = GROmerge.append(per_imp, ignore_index=True)
GROmerge

In [ ]:
cols = list(GROmerge)
cols

In [ ]:
GROmerge['Percent row imputed'] = GROmerge.isnull().sum(axis=1) / (len(cols) - 2)
## subtract 2 to only count numeric cols
GROmerge


In [ ]:
GROmerge['Renewable energy_imputed'] = np.nan

for index, row in GROmerge.iterrows():
    if np.isnan(row['Renewable energy consumption (TJ)']):
        #print(row['Country'])
        GROmerge.at[index, 'Renewable energy_imputed'] = df.iloc[index]['Renewable energy consumption (TJ)']

#def newcol(x):
   #if np.isnan(x['Renewable energy consumption (TJ)']):
        #return df['Renewable energy consumption (TJ)']

#GROmerge['Renewable energy_imputed'] = GROmerge.apply(newcol, axis=1)
#GROmerge['Renewable energy_imputed']

In [ ]:
GROmerge

In [ ]:
GROmerge['Tree cover_imputed'] = np.nan

for index, row in GROmerge.iterrows():
    if np.isnan(row['Annual tree cover loss (ha)']):
        GROmerge.at[index, 'Tree cover_imputed'] = df.iloc[index]['Annual tree cover loss (ha)']


In [ ]:
GROmerge

In [ ]:
GROmerge['Fossil fuel GDP_imputed'] = np.nan

for index, row in GROmerge.iterrows():
    if np.isnan(row['Fossil fuel subsidies as % GDP']):
        GROmerge.at[index, 'Fossil fuel GDP_imputed'] = df.iloc[index]['Fossil fuel subsidies as % GDP']
GROmerge

In [ ]:
GROmerge['Fossil fuel per cap_imputed'] = np.nan
GROmerge['Air pollution_imputed'] = np.nan

for index, row in GROmerge.iterrows():
    if np.isnan(row['Fossil fuel subsidies USD per Capita']):
        GROmerge.at[index, 'Fossil fuel per cap_imputed'] = df.iloc[index]['Fossil fuel subsidies USD per Capita']
    if np.isnan(row['Air pollution (annual mean ug/m3 <PM2.5)']):   
        GROmerge.at[index, 'Air pollution_imputed'] = df.iloc[index]['Air pollution (annual mean ug/m3 <PM2.5)']
GROmerge

In [ ]:
GROmerge['Renewable_final'] = GROmerge['Renewable energy consumption (TJ)'].fillna(GROmerge['Renewable energy_imputed'])
GROmerge

In [ ]:
GROmerge['Tree cover_final'] = GROmerge['Annual tree cover loss (ha)'].fillna(GROmerge['Tree cover_imputed'])
GROmerge['Fossil GDP_final'] = GROmerge['Fossil fuel subsidies as % GDP'].fillna(GROmerge['Fossil fuel GDP_imputed'])
GROmerge['Fossil per cap_final'] = GROmerge['Fossil fuel subsidies USD per Capita'].fillna(GROmerge['Fossil fuel per cap_imputed'])
GROmerge['Air poll_final'] = GROmerge['Air pollution (annual mean ug/m3 <PM2.5)'].fillna(GROmerge['Air pollution_imputed'])
GROmerge

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=True)
model.fit(GROmerge[['Tree cover_final', 'Fossil GDP_final', 'Fossil per cap_final', 'Air poll_final',
                    'Renewable_final', 'Natural resource rents as % GDP', 'CO2 emissions tons per capita']], GROmerge['Percent'])

In [ ]:
# line of best fit calculated and saved into model
#access the parameters in the model (gradient and y intercept)
#model.predict()

GROfinal = GROmerge[['Tree cover_final', 'Fossil GDP_final', 'Fossil per cap_final', 'Air poll_final',
                    'Renewable_final', 'Natural resource rents as % GDP', 'CO2 emissions tons per capita', 'Percent', 'Green recovery spending']]
GROfinal

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(GROfinal, alpha=0.5, figsize=(17, 17), diagonal='kde')

In [ ]:
corr = GROfinal.corr(method='pearson')

In [ ]:
#remember this is after imputation
import matplotlib.pyplot as plt
import seaborn as sns

#colourmap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, cmap=cmap, annot=True)

In [ ]:
model.coef_

In [ ]:
#line of best fit parameters access

model.intercept_

In [ ]:
#ax = GROfinal.plot.scatter("Renewable_final", "Green recovery spending")
plt.scatter(GROfinal["Renewable_final"], GROfinal["Green recovery spending"])

In [ ]:
model = LinearRegression(fit_intercept=True)

In [ ]:
model.fit(GROfinal[['Renewable_final']], GROfinal['Green recovery spending'])

In [ ]:
data = {'Vals': [GROfinal['Renewable_final'].min(), GROfinal['Renewable_final'].max()]}
xfit = pd.DataFrame(data, columns = ['Vals'])
#xfit = DataFrame([GROfinal['Renewable_final'].min(), GROfinal['Renewable_final'].max()])
ypred = model.predict(xfit)

In [ ]:
#ax.plot(xfit, ypred, linestyle="dashed")
plt.scatter(GROfinal["Renewable_final"], GROfinal["Green recovery spending"])
plt.plot(xfit, ypred, linestyle="dashed", color='red')
#limited linear correlation

In [ ]:
GROfinal.shape

In [ ]:
#kmeans clustering - unsupervised no target

#not ff twice Fossil GDP_final
from sklearn.cluster import KMeans
GROxcols = GROfinal[['Tree cover_final','Fossil per cap_final', 'Air poll_final',
                    'Renewable_final', 'Natural resource rents as % GDP', 'CO2 emissions tons per capita']]

In [ ]:
#normalise the data
scaler = MinMaxScaler()
GROxscaled = pd.DataFrame(scaler.fit_transform(GROxcols), columns = GROxcols.columns)
GROxscaled = GROxscaled.drop(index=42)
GROxscaled

In [ ]:
#how are these related to each other
#put them all into categories
kmeans = KMeans(n_clusters=3).fit(GROxscaled)

In [ ]:
scatter_matrix(GROxscaled, figsize=(17,17), c=kmeans.labels_)
#set colour based on calculated labels

In [ ]:
cc = pd.DataFrame(kmeans.cluster_centers_, columns = ['Tree cover_final','Fossil per cap_final', 'Air poll_final',
                    'Renewable_final', 'Natural resource rents as % GDP', 'CO2 emissions tons per capita'])
cc
#shows point for each feature for each cluster (6 features, 3 clusters)

In [ ]:
GROxcols = GROxcols.drop(index=42)
GROxcols['Labels_3'] = kmeans.labels_

In [ ]:
from sklearn.datasets import make_blobs

inertia_values = []
r = pd.RangeIndex(2, 8)
for n_clusters in r:
    kmeans = KMeans(n_clusters=n_clusters).fit(GROxscaled)
    inertia_values.append(kmeans.inertia_)
    
inertia = pd.Series(inertia_values, name="inertia", index=r)
inertia.plot()
#elbows around 3, use that many clusters
#lower number is better fit - 3, 4, 7?

In [ ]:
GROxcols['Labels_7'] = kmeans.labels_

In [ ]:
GROxcols
#which cluster is each sample in given a k of 7?